# Прогноз стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

Импортируем все необходимые библиотеки в следующей ячейке:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import math
import time
import warnings
warnings.simplefilter("ignore")

Откроем датасет:

In [2]:
df = pd.read_csv('/datasets/autos.csv')
df

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

В датасете много пропусков, сперва заполним столбец NotRepaired. Данный столбец как судя по всему все остальные имеющие пропуски заполняется вручную, и скорее всего столбец ремонта не заполнили намеренно, так как ремонт машины существенно влияет на стоимость, и некоторые пользователи могли хотеть скрыть этот факт, на мой взгляд лучше будет указать в нем not specified, для того чтобы модель и такие данные брала для предсказания.

In [4]:
df['NotRepaired'] = df['NotRepaired'].fillna('not specified')
df['NotRepaired'].value_counts()

no               247161
not specified     71154
yes               36054
Name: NotRepaired, dtype: int64

Далее удалим пустые значений в столбце Model, так как этот параметр имеет наибольший вес при оценке предсказаний будет не целесообразно заполнять пустые значения. Так же после заполним столбцы VehicleType значениями other, а так же Gearbox значениями auto. Gearbox заполняем значениями auto, так как скорее всего пропуски здесь остались потому что клиенты не знали что указать имея в машине коробку робот или вариатор, но будем считать их разновидностью автомата и заполним столбец значениями auto. 

In [5]:
df= df.dropna(subset=['Model'])
df= df.dropna(subset=['Power'])
df['VehicleType'] = df['VehicleType'].fillna('other')
df['Gearbox'] = df['Gearbox'].fillna('auto')
df['FuelType'].value_counts()

petrol      207162
gasoline     95857
lpg           4962
cng            544
hybrid         208
other          133
electric        66
Name: FuelType, dtype: int64

Судя по всему, бензин записан в двух формах: petrol, gasoline. Лучше будет привести их к одной, и так как бензин все-таки преобладает как вид топлива пропуски лучше заполнить им:

In [6]:
df.loc[df['FuelType'] == 'gasoline', 'FuelType'] = 'petrol'
df['FuelType'] = df['FuelType'].fillna('petrol')
df 

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,other,1993,manual,0,golf,150000,0,petrol,volkswagen,not specified,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,petrol,jeep,not specified,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,petrol,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354363,2016-03-27 20:36:20,1150,bus,2000,manual,0,zafira,150000,3,petrol,opel,no,2016-03-27 00:00:00,0,26624,2016-03-29 10:17:23
354364,2016-03-21 09:50:58,0,other,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,petrol,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


Далее удалим столбец DateCrawled он полностью соответствует столбцу DateCreated, к тому же даже если этот параметр и отличается то не думаю что он как-то повлияет на итоговую оценку стоиомсти авто. Аналогично поступим и с LastSeen и DateCreated, так как посещаемость пользователя не влияет на стоимость авто как и дата публикации объявления. 

In [7]:
df = df.drop(columns = ['DateCrawled'],axis = 1)
df = df.drop(columns = ['LastSeen'],axis = 1)
df = df.drop(columns = ['DateCreated'],axis = 1)
df['NumberOfPictures'].value_counts()

0    334664
Name: NumberOfPictures, dtype: int64

In [8]:
df['Price'].value_counts()

0        8588
500      5215
1500     5001
1200     4299
1000     4251
         ... 
3988        1
14227       1
11920       1
1429        1
8188        1
Name: Price, Length: 3670, dtype: int64

Похоже в столбце NumberOfPictures все значения нули, поэтому будет лучше убрать этот столбец, так как он никакой роли не сыграет. Так же сразу удалим столбец PostalCode, им возможно было бы воспользоваться, если бы знали о каком регионе или стране идет речь, но так как у нас нет такой информации и мы не можем понять регион, то его лучше удалить. Также судя по всему в столбце таргетов price есть нулевой показатель цены, возможно клиенты хотели чтобы им позвонили и после обговорить цену с возможным покупателем, так как в этом случае его возможно будет легче уговорить, в этом случае цена может быть завышена по отношению к другим авто такой же модели, поэтому пытаться заполнить нули ценой из других объявлений с такими же машинами не совсем корректно, потому что это может негативно сказаться на прогнозе модели, поэтому эти данные лучше удалить.

In [9]:
df = df.drop(columns = ['NumberOfPictures'],axis = 1)
df = df.drop(columns = ['PostalCode'],axis = 1)
df= df.query('Price != 0')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326076 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              326076 non-null  int64 
 1   VehicleType        326076 non-null  object
 2   RegistrationYear   326076 non-null  int64 
 3   Gearbox            326076 non-null  object
 4   Power              326076 non-null  int64 
 5   Model              326076 non-null  object
 6   Kilometer          326076 non-null  int64 
 7   RegistrationMonth  326076 non-null  int64 
 8   FuelType           326076 non-null  object
 9   Brand              326076 non-null  object
 10  NotRepaired        326076 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.9+ MB


Ноли также встречаются в столбце Power, мощность авто может быть равной нулю, возможны варианты когда машина на разбор или она без двигателя, модели нужно также определять и предсказывать такие варианты.

Далее проверим год выпуска:

In [12]:
sorted(df['RegistrationYear'].unique())

[1000,
 1001,
 1234,
 1300,
 1400,
 1500,
 1600,
 1602,
 1800,
 1910,
 1923,
 1927,
 1928,
 1929,
 1930,
 1931,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1938,
 1941,
 1942,
 1943,
 1945,
 1947,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2066,
 2200,
 2290,
 2500,
 2900,
 3000,
 3700,
 4000,
 4500,
 5000,
 5555,
 5900,
 5911,
 6000,
 6500,
 7000,
 7100,
 7800,
 8000,
 8200,
 8500,
 9000,
 9999]

Судя по всему пользователи заполняли год как захочется, нужно понять сколько проблемных данных и скажутся они на показателях или нет, границами будут 1910 и 2016 год, в этих пределах данные кажутся наиболее достоверными, так как до 1910 года идут даты в которые машин еще просто не было, а среди объявлений вполне могли попасться раритеты, а после 2016 данные не актуальны для датасета, так как самая поздняя регистрация объявления была в 2016.

In [13]:
print(df[df['RegistrationYear'] < 1910].count())
print(df[df['RegistrationYear'] > 2016].count())

Price                28
VehicleType          28
RegistrationYear     28
Gearbox              28
Power                28
Model                28
Kilometer            28
RegistrationMonth    28
FuelType             28
Brand                28
NotRepaired          28
dtype: int64
Price                11920
VehicleType          11920
RegistrationYear     11920
Gearbox              11920
Power                11920
Model                11920
Kilometer            11920
RegistrationMonth    11920
FuelType             11920
Brand                11920
NotRepaired          11920
dtype: int64


Данных много, лучше будет их заменить на медианные значения годов выпуска в зависимости от модели.

In [14]:
year = df[df['RegistrationYear'] != 0]
year = year.groupby('Model')['RegistrationYear'].median().reset_index().rename(columns = {'RegistrationYear': 'RegistrationYear_mean'})
year['RegistrationYear_mean'] = year['RegistrationYear_mean'].astype('int')
df = pd.merge(df, year, on='Model', how = 'left')  
df.loc[df['RegistrationYear'] > 2016, ['RegistrationYear']] = df['RegistrationYear_mean']
df.loc[df['RegistrationYear'] < 1910, ['RegistrationYear']] = df['RegistrationYear_mean']
df = df.drop(columns = ['RegistrationYear_mean'],axis = 1)
df['RegistrationYear'] = df['RegistrationYear'].astype('int')

In [15]:
sorted(df['RegistrationYear'].unique())

[1910,
 1923,
 1927,
 1928,
 1929,
 1930,
 1931,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1938,
 1941,
 1942,
 1943,
 1945,
 1947,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016]

Далее преобразуем датасет и создадим отдельный датасет для логистической регрессии и для буст моделей:

In [16]:
df_gd = df
df = pd.get_dummies(df, drop_first=True)
df

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,...,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_not specified,NotRepaired_yes
0,480,1993,0,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
1,9800,2004,163,125000,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1500,2001,75,150000,6,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,3600,2008,69,90000,7,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,650,1995,102,150000,10,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326071,3200,2004,225,150000,5,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
326072,1150,2000,0,150000,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326073,1199,2000,101,125000,3,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
326074,9200,1996,102,150000,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Разделим датасет на выборки:

In [17]:
target = df['Price']
features = df.drop('Price', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
target = df_gd['Price']
features = df_gd.drop('Price', axis=1)
features_train_gd, features_valid_gd, target_train_gd, target_valid_gd = train_test_split(
    features, target, test_size=0.25, random_state=12345)


Масштабируем признаки:

In [18]:
numeric = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train_gd[numeric] = scaler.transform(features_train_gd[numeric])
features_valid_gd[numeric] = scaler.transform(features_valid_gd[numeric])

**Вывод** В данных:
* заменены значения NaN;
* убраны ненужные для машинного обучения столбцы;
* применен метод OHE;
* датасет разделен на выборки;
* произведено масштабирование.

Теперь выборки готовы для обучения моделей.

## Обучение моделей

### Логистическая регрессия

In [19]:
%%time
start = time.time()
model_lr = LinearRegression()
model_lr.fit(features_train, target_train) 
end = time.time()
time_lr = end - start
start = time.time()
predictions_valid_lr = model_lr.predict(features_valid) 
result_lr = mean_squared_error(target_valid, predictions_valid_lr, squared=False)
end = time.time()
time_lr_pred = end - start
print('rmse =', result_lr)

rmse = 2847.6545262508203
CPU times: user 37.6 s, sys: 42.2 s, total: 1min 19s
Wall time: 1min 19s


### Catboost

Далее сделаем собственный скорер для gridsearch:

In [20]:
def rmse(target, predict):
    target = np.array(target)
    predict = np.array(predict)
    return np.sqrt(((predictions - targets) ** 2).mean())
rmse_s = make_scorer(rmse, greater_is_better=False)
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [21]:
parametrs = {'n_estimators': range (10, 51, 10),
              'max_depth': range (1,13, 2)            
}
catboost = CatBoostRegressor(random_state = 12345, cat_features = cat_features)
grid_cb = GridSearchCV(catboost, parametrs, cv=5, scoring = rmse_s)
grid_cb.fit(features_train_gd, target_train_gd)
grid_cb.best_params_

Learning rate set to 0.5
0:	learn: 3883.0799621	total: 102ms	remaining: 915ms
1:	learn: 3538.7001972	total: 140ms	remaining: 559ms
2:	learn: 3321.1835205	total: 195ms	remaining: 455ms
3:	learn: 3184.0245692	total: 229ms	remaining: 344ms
4:	learn: 3077.2736993	total: 274ms	remaining: 274ms
5:	learn: 2970.1016320	total: 303ms	remaining: 202ms
6:	learn: 2897.6173775	total: 339ms	remaining: 145ms
7:	learn: 2824.8723057	total: 378ms	remaining: 94.5ms
8:	learn: 2769.4045008	total: 415ms	remaining: 46.1ms
9:	learn: 2711.3422053	total: 459ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 3877.2477909	total: 40.2ms	remaining: 362ms
1:	learn: 3532.7677657	total: 69.7ms	remaining: 279ms
2:	learn: 3314.6445296	total: 116ms	remaining: 271ms
3:	learn: 3177.2732842	total: 144ms	remaining: 216ms
4:	learn: 3070.9752090	total: 184ms	remaining: 184ms
5:	learn: 2968.7189725	total: 228ms	remaining: 152ms
6:	learn: 2897.3840075	total: 261ms	remaining: 112ms
7:	learn: 2824.3912832	total: 295ms	remaining: 

{'max_depth': 1, 'n_estimators': 10}

In [22]:
%%time
start = time.time()
catboost = CatBoostRegressor(random_state = 12345, cat_features = cat_features, loss_function = 'RMSE', iterations = 500)
catboost.fit(features_train_gd, target_train_gd, verbose = 100)
end = time.time()
time_cb = end - start
start = time.time()
predictions_valid_cb = catboost.predict(features_valid_gd) 
result_cb = mean_squared_error(target_valid_gd, predictions_valid_cb)**0.5
end = time.time()
time_cb_pred = end - start
print('rmse =', result_cb)

Learning rate set to 0.171507
0:	learn: 4034.0833587	total: 330ms	remaining: 2m 44s
100:	learn: 1751.1664365	total: 24.3s	remaining: 1m 36s
200:	learn: 1675.0330972	total: 47.9s	remaining: 1m 11s
300:	learn: 1638.6991248	total: 1m 12s	remaining: 47.9s
400:	learn: 1610.4895601	total: 1m 39s	remaining: 24.5s
499:	learn: 1591.3018127	total: 2m 5s	remaining: 0us
rmse = 1659.9391200352848
CPU times: user 2min 6s, sys: 633 ms, total: 2min 7s
Wall time: 2min 9s


### LightGBM

In [27]:
features_train_gd[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']] = features_train_gd[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']].astype('category')
features_valid_gd[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']] = features_valid_gd[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']].astype('category')
features_train_gd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244557 entries, 197805 to 217570
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   VehicleType        244557 non-null  category
 1   RegistrationYear   244557 non-null  float64 
 2   Gearbox            244557 non-null  category
 3   Power              244557 non-null  float64 
 4   Model              244557 non-null  category
 5   Kilometer          244557 non-null  float64 
 6   RegistrationMonth  244557 non-null  float64 
 7   FuelType           244557 non-null  category
 8   Brand              244557 non-null  category
 9   NotRepaired        244557 non-null  category
dtypes: category(6), float64(4)
memory usage: 11.0 MB


In [24]:
parametrs = {'n_estimators': range (10, 51, 10),
              'max_depth': range (1,13, 2)            
}
LGBM = LGBMRegressor(random_state = 12345)
grid_lgbm = GridSearchCV(LGBM, parametrs, cv=5, scoring=rmse_s)
grid_lgbm.fit(features_train_gd, target_train_gd)
grid_lgbm.best_params_

{'max_depth': 1, 'n_estimators': 10}

In [36]:
%%time
start = time.time()
model_lgbm = LGBMRegressor(random_state = 12345, max_depth = 1, n_estimators =10, metrics = 'rmse_s')
model_lgbm.fit(features_train_gd, target_train_gd)
end = time.time()
time_lgbm = end - start
start = time.time()
predictions_valid_lgbm = model_lgbm.predict(features_valid_gd) 
result_lgbm = mean_squared_error(target_valid_gd, predictions_valid_lgbm)**0.5
end = time.time()
time_lgbm_pred = end - start
print('rmse =', result_lgbm)

rmse = 3617.4344295606793
CPU times: user 4.72 s, sys: 43.9 ms, total: 4.77 s
Wall time: 4.7 s


**Вывод**

Лучшей по показателю rmse оказалась модель catboost.

## Анализ моделей

Соберем результаты в один датасет:

In [31]:
analysis = {'RMSE' : pd.Series([result_lr, result_cb, result_lgbm], index = ['LinearRegression', 'CatBoostRegressor', 'LGBMRegressor']),
            'Time' : pd.Series([time_lr, time_cb, time_lgbm], index = ['LinearRegression', 'CatBoostRegressor', 'LGBMRegressor']),
            'Time_pred' : pd.Series([time_lr_pred, time_cb_pred, time_lgbm_pred], index = ['LinearRegression', 'CatBoostRegressor', 'LGBMRegressor'])
}
df_analysis = pd.DataFrame(analysis)
df_analysis

,RMSE,Time,Time_pred
LinearRegression,2847.654526,79.754377,0.202178
CatBoostRegressor,1659.939120,128.661971,0.464828
LGBMRegressor,3617.434430,4.229302,0.105499


**Вывод**

Лучшим по показателям RMSE является catboost, но при этом у этой модели самое долгое обучения и предсказание, далее идет линейная регрессия она имеет средние показатели, а после идет модель LGBM у неё самый низкий показатель RMSE, но при этому самая большая скорость.